### 実装方針
#### 1. タグ候補を出す
形態素解析で「名詞」かつ「一般」の単語を取り出す。この時、出現回数が多いものと1つしかないものは取り除く

#### 2. 全文書からタグ候補以外を削除する
タグを付ける上ではノイズにしかならないので取り除く

#### 3. タグ候補を取り除いたもの同士で、類似文書を取得する
文書をTF-IDFでベクトル化し、コサイン類似度を計算する。
類似している文章をそれぞれ5つずつ取り出す

#### 4. 類似している文章を全て足し合わせ、そこで使用回数が多いものをタグとする
足し合わせることで下記のメリットがある
- 表記揺れに強くなる
- 文書に存在していない単語のタグ付けができる

例えば１つの文書だけでタグ付けを判断してしまうと、「車」というタグがつくものと、「自動車」というタグがつくものにわかれてしまう可能性があるが、似た文書の和を先にとっておくことで、どちらのタグもつけることができる

In [ ]:
# ライブラリ読み込み
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import MeCab
import matplotlib.pyplot as plt

# CSV読み込み
path="csv/"
train = pd.read_csv(path + 'train.csv')

# MeCab
m = MeCab.Tagger ("-Ochasen")

In [ ]:
# [実験]MeCab 「名詞」の「一般」のみ取得する
print(m.parse("天皇陛下の即位と同時に元号が平成から「令和」に改まった"))

In [ ]:
# 名詞かつ一般を抽出する関数
def extract_nouns(text):
    node = m.parseToNode(text)
    nouns = []
     
    while node:
        features =  node.feature.split(',')
         
        if features[0] == '名詞' and features[1] == '一般':
            nouns.append(node.surface)
         
        node = node.next
     
    return set(nouns)

# titleも含めて名詞を抽出する
train['text'] = train['title'] + train['content']
train['words'] = train['text'].apply(lambda x: extract_nouns(str(x)))

In [ ]:
# 単語の出現頻度確認
from collections import defaultdict

unigrams = defaultdict(int)

for row in train['words']:
    for word in row:  
      unigrams[word] += 1
    
df_unigrams = pd.DataFrame(sorted(unigrams.items(), key=lambda x: x[1])[::-1])
df_unigrams

In [ ]:
# 単語の出現頻度確認（グラフ）
import matplotlib

font = {"family": "TakaoGothic"}
matplotlib.rc('font', **font)

igfont = {'family':'IPAexGothic'}
plt.title('出現回数比率（1~100）', **igfont)
plt.hist(df_unigrams[1], range=(1,100), bins=16, rwidth=0.8)

In [ ]:
#  名詞かつ一般でないものは取り除いて分かち書きにする
def wakati(text):
    node = m.parseToNode(text)
    wakati_content = ''
    
    while node:
        features =  node.feature.split(',')
        
        if features[0] == '名詞' and features[1] == '一般':
          wakati_content = wakati_content + node.surface + ' '
        
        node = node.next
    
    return wakati_content

train['wakati'] = train['text'].apply(lambda x: wakati(str(x)))
train['wakati']

In [ ]:
# 出現回数は1が圧倒的に多い。1回だとタグ付に適さないので取り除く
# 出現回数が多いものについても、タグとして適さないと考えられるので取り除く
# ここでは出現回数が50回以下かつ2回以上をタグ候補として残す
df_tags = pd.DataFrame(df_unigrams.loc[df_unigrams[1] <= 50])
df_tags = pd.DataFrame(df_tags.loc[df_unigrams[1] >= 2])

np_tags = np.array(df_tags[0])
np_tags

In [ ]:
# タグ候補の単語のみ各文章に残す
# 実行時間は30秒ほど
def remove_unselectable_tags(text):
    train_words = text.split()
    filtered_text = ''
    
    for word in train_words:
      for tag_word in np_tags:
        if word == tag_word:
            filtered_text =  filtered_text + word + ' '
    
    return filtered_text


train['wakati'] = train['wakati'].apply(lambda x: remove_unselectable_tags(str(x)))

In [ ]:
# ライブラリ読み込み
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tf = vectorizer.fit_transform(train['wakati']) 
tfidf = transformer.fit_transform(tf) 
tfidf_array = tfidf.toarray()
cs = cosine_similarity(tfidf_array, tfidf_array)

In [ ]:
# TF-IDFからのコサイン類似度で文書をクラスタリング
docs = train['wakati']

indexs = []
for (i, c) in enumerate(cs):
    c_np = np.array(c)
    c_sort = np.sort(c_np)
    index1 = np.where(c_np == c_sort[-2])[0][0]
    index2 = np.where(c_np == c_sort[-3])[0][0]
    index3 = np.where(c_np == c_sort[-4])[0][0]
    index4 = np.where(c_np == c_sort[-5])[0][0]
    index5 = np.where(c_np == c_sort[-6])[0][0]
    indexs.append([
                   i,
                   '"' + docs[i] + '"',
                   '"' + docs[index1] + '"',
                   '"' + docs[index2] + '"',
                   '"' + docs[index3] + '"',
                   '"' + docs[index4] + '"',
                   '"' + docs[index5] + '"',
                   c_sort[-2], c_sort[-3], c_sort[-4], c_sort[-5], c_sort[-6]
                   ])

df_result = pd.DataFrame(indexs, columns=['id', 'target', 'top1', 'top2', 'top3', 'top4', 'top5', 'score1', 'score2', 'score3', 'score4', 'score5'])
df_result

In [ ]:
# 使用頻度が高い言葉を取り出す関数
def count_tags(text):
    contents = text.split()
    
    tag_counts = defaultdict(int)

    for word in contents:
      tag_counts[word] += 1
    

    tag_counts = sorted(tag_counts.items(), key=lambda x: x[1], reverse=True)
    
    # タグを5つままで出力する。ただしcountが閾値を超えているもののみとする
    threshold = 5
    tags = []
    cnt = 0
    for i in range(len(tag_counts)):
        if cnt > 4:
            break
        if tag_counts[i][1] >= threshold:
            tags.append(tag_counts[i][0])
            cnt += 1
            

    return tags

In [ ]:
# 類似している5つの文書を足し合わせ、使用頻度が高い上位5つの言葉をタグとする
df_result['candidates'] = df_result['target']+df_result['top1']+df_result['top2']+df_result['top3'] + df_result['top4'] + df_result['top5']
df_result['tags' ] = df_result['candidates'].apply(lambda x: count_tags(str(x)))
df_result['tags']

In [ ]:
# 提出用データ生成
df_sub = pd.read_csv(path + 'train.csv')

def get_tag_by_index(tags, index):    
    if len(tags) < index:
        return ''
    else:
        return tags[index-1]
    
df_sub['tag1'] = df_result['tags'].apply(lambda x: get_tag_by_index(x,1))
df_sub['tag2'] = df_result['tags'].apply(lambda x: get_tag_by_index(x,2))
df_sub['tag3'] = df_result['tags'].apply(lambda x: get_tag_by_index(x,3))
df_sub['tag4'] = df_result['tags'].apply(lambda x: get_tag_by_index(x,4))
df_sub['tag5'] = df_result['tags'].apply(lambda x: get_tag_by_index(x,5))
df_sub

In [ ]:
df_sub.to_csv("csv/submission.csv", encoding='utf_8_sig')

### 今後の課題
#### 1. パラメータの最適化
今回パラメータが出てくるのは下記3箇所
- タグ候補 
- 似た文書をいくつ足すか
- タグを最終的に決定するための閾値

現在は感覚で各値を設定しているが、もうちょっといい方法がないか検討したい

#### 2. 別の形態素解析器を使う
「暗号通貨」という言葉が「暗号」と「通貨」　のようにわかれてしまっているのでそこを改善したい

#### 3. 別の手法での表記ゆれ対応
今回は類似文章の和を取ることで、表記ゆれに対応したが、LSI(潜在意味解析)というのものも使ってみたい